In [21]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.optimizers import Adam

import pickle
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import os
import csv
from PIL import Image


In [22]:
(train_set, train_label_raw), (X_test, y_test) = mnist.load_data()
train_label = to_categorical(train_label_raw)
train_data = np.asarray(train_set).reshape(60000, 28, 28, 1)

In [23]:
gen_images_raw = pd.read_csv('generated_images_47600_0.923000.csv', header=None)
X_fake = np.asarray(gen_images_raw).reshape(100000, 28, 28, 1)
gen_labels_raw = pd.read_csv('generated_labels_47600_0.923000.csv', header=None)
y_fake = to_categorical(gen_labels_raw)

In [50]:
class classifier():
    def __init__(self):
        
        self.model = Sequential()
        # model.add(Conv2D(5, kernel_size=3, activation='relu', input_shape=(28,28,1)))
        self.model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))
        self.model.add(MaxPooling2D(pool_size=(4, 4)))
        self.model.add(Conv2D(32, kernel_size=3, activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Flatten())
        self.model.add(Dense(10, activation='softmax'))

        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    def train(self, totTrain=100_000, epochs=10, ratio=(1,1)): #Real to Fake
        sum = ratio[0] + ratio[1]
        realTrain = ( totTrain // sum ) * ratio[0] 
        fakeTrain = ( totTrain // sum ) * ratio[1] 

        X_train_raw, X_val, y_train_raw, y_val = train_test_split(train_data, train_label, test_size=0.33, random_state=43)

        X_train = np.concatenate((X_train_raw[:realTrain], X_fake[:fakeTrain]))
        y_train = np.concatenate((y_train_raw[:realTrain], y_fake[:fakeTrain]))

        print(X_train.shape)
        print(X_train_raw[:realTrain].shape)
        print(X_fake[:fakeTrain].shape)

        # model.summary()

        model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs)
        os.makedirs('classif_history/', exist_ok=True)

        root = 'classif_history/'
        with open(root + 'classif_'+ str(totTrain) + '_' + str(ratio[0]) + '_' + str(ratio[1]) + '.csv', 'w') as f:
            for key in model.history.history.keys():
                f.write("%s,%s\n"%(key,model.history.history[key]))

        X_test_reshaped = X_test.reshape(10000, 28, 28, 1)
        loss, acc = model.evaluate(X_test_reshaped, to_categorical(y_test))
        print(acc)
        with open(root + 'classif_acc.csv', 'a') as b:
            b.write(str(totTrain) + ',' + str(ratio[0]) + ',' + str(ratio[1]) + ',' + "%f\n"%(acc))
            

In [52]:
ratio_array = [  (0,1), (1,0), (1,1), (2,1), (1,2), (3,1), (1,3), (5,1), (1,5), (10,1), (1,10)]
train_array = [ 1000, 5000, 10000, 50000 ]



for num in train_array:
    for ratio in ratio_array:
        if (ratio != ratio_array[0]) & (num != train_array[0]):
            del cl
        cl = classifier()
        cl.train(totTrain=num, epochs=25, ratio=ratio)


(1000, 28, 28, 1)
(0, 28, 28, 1)
(1000, 28, 28, 1)
Train on 1000 samples, validate on 19800 samples
Epoch 1/25
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0120 - acc: 0.9970 - val_loss: 0.4092 - val_acc: 0.9526
Epoch 2/25
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0065 - acc: 0.9980 - val_loss: 0.3890 - val_acc: 0.9537
Epoch 3/25
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0022 - acc: 0.9980 - val_loss: 0.3829 - val_acc: 0.9552
Epoch 4/25
1000/1000 [==============================] - 2s 2ms/step - loss: 9.6834e-05 - acc: 1.0000 - val_loss: 0.3784 - val_acc: 0.9565
Epoch 5/25
1000/1000 [==============================] - 2s 2ms/step - loss: 5.5165e-05 - acc: 1.0000 - val_loss: 0.3783 - val_acc: 0.9565
Epoch 6/25
1000/1000 [==============================] - 2s 2ms/step - loss: 4.0564e-05 - acc: 1.0000 - val_loss: 0.3783 - val_acc: 0.9565
Epoch 7/25
1000/1000 [==============================] - 2s 2ms/step - loss: 3.3715e-

Epoch 8/25
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0323 - acc: 0.9980 - val_loss: 0.3176 - val_acc: 0.9612
Epoch 9/25
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0323 - acc: 0.9980 - val_loss: 0.3170 - val_acc: 0.9614
Epoch 10/25
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0323 - acc: 0.9980 - val_loss: 0.3167 - val_acc: 0.9614
Epoch 11/25
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0323 - acc: 0.9980 - val_loss: 0.3165 - val_acc: 0.9614
Epoch 12/25
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0323 - acc: 0.9980 - val_loss: 0.3163 - val_acc: 0.9614
Epoch 13/25
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0323 - acc: 0.9980 - val_loss: 0.3161 - val_acc: 0.9615
Epoch 14/25
1000/1000 [==============================] - 2s 2ms/step - loss: 0.0323 - acc: 0.9980 - val_loss: 0.3159 - val_acc: 0.9615
Epoch 15/25
1000/1000 [==============================] - 

Epoch 17/25
999/999 [==============================] - 2s 2ms/step - loss: 0.0161 - acc: 0.9990 - val_loss: 0.3116 - val_acc: 0.9626
Epoch 18/25
999/999 [==============================] - 2s 2ms/step - loss: 0.0161 - acc: 0.9990 - val_loss: 0.3117 - val_acc: 0.9625
Epoch 19/25
999/999 [==============================] - 2s 2ms/step - loss: 0.0161 - acc: 0.9990 - val_loss: 0.3117 - val_acc: 0.9625
Epoch 20/25
999/999 [==============================] - 2s 2ms/step - loss: 0.0161 - acc: 0.9990 - val_loss: 0.3117 - val_acc: 0.9625
Epoch 21/25
999/999 [==============================] - 2s 2ms/step - loss: 0.0161 - acc: 0.9990 - val_loss: 0.3117 - val_acc: 0.9625
Epoch 22/25
999/999 [==============================] - 2s 2ms/step - loss: 0.0161 - acc: 0.9990 - val_loss: 0.3117 - val_acc: 0.9625
Epoch 23/25
999/999 [==============================] - 2s 2ms/step - loss: 0.0161 - acc: 0.9990 - val_loss: 0.3117 - val_acc: 0.9625
Epoch 24/25
999/999 [==============================] - 2s 2ms/step - 

10000/10000 [==============================] - 1s 80us/step
0.9681
(996, 28, 28, 1)
(830, 28, 28, 1)
(166, 28, 28, 1)
Train on 996 samples, validate on 19800 samples
Epoch 1/25
996/996 [==============================] - 2s 2ms/step - loss: 0.0324 - acc: 0.9980 - val_loss: 0.3127 - val_acc: 0.9628
Epoch 2/25
996/996 [==============================] - 2s 2ms/step - loss: 0.0324 - acc: 0.9980 - val_loss: 0.3125 - val_acc: 0.9628
Epoch 3/25
996/996 [==============================] - 2s 2ms/step - loss: 0.0324 - acc: 0.9980 - val_loss: 0.3125 - val_acc: 0.9629
Epoch 4/25
996/996 [==============================] - 2s 2ms/step - loss: 0.0324 - acc: 0.9980 - val_loss: 0.3124 - val_acc: 0.9629
Epoch 5/25
996/996 [==============================] - 2s 2ms/step - loss: 0.0324 - acc: 0.9980 - val_loss: 0.3124 - val_acc: 0.9629
Epoch 6/25
996/996 [==============================] - 2s 2ms/step - loss: 0.0324 - acc: 0.9980 - val_loss: 0.3124 - val_acc: 0.9630
Epoch 7/25
996/996 [======================

990/990 [==============================] - 2s 2ms/step - loss: 0.0707 - acc: 0.9939 - val_loss: 0.4692 - val_acc: 0.9599
Epoch 9/25
990/990 [==============================] - 2s 2ms/step - loss: 0.0904 - acc: 0.9939 - val_loss: 0.4641 - val_acc: 0.9605
Epoch 10/25
990/990 [==============================] - 2s 2ms/step - loss: 0.0684 - acc: 0.9929 - val_loss: 0.4382 - val_acc: 0.9619
Epoch 11/25
990/990 [==============================] - 2s 2ms/step - loss: 0.0488 - acc: 0.9970 - val_loss: 0.4184 - val_acc: 0.9635
Epoch 12/25
990/990 [==============================] - 2s 2ms/step - loss: 0.0489 - acc: 0.9970 - val_loss: 0.4192 - val_acc: 0.9634
Epoch 13/25
990/990 [==============================] - 2s 2ms/step - loss: 0.0488 - acc: 0.9970 - val_loss: 0.4193 - val_acc: 0.9634
Epoch 14/25
990/990 [==============================] - 2s 2ms/step - loss: 0.0488 - acc: 0.9970 - val_loss: 0.4193 - val_acc: 0.9634
Epoch 15/25
990/990 [==============================] - 2s 2ms/step - loss: 0.0488 

...

40200/40200 [==============================] - 9s 230us/step - loss: 0.4955 - acc: 0.9689 - val_loss: 0.4944 - val_acc: 0.9690
Epoch 7/25
40200/40200 [==============================] - 9s 234us/step - loss: 0.4820 - acc: 0.9698 - val_loss: 0.4840 - val_acc: 0.9696
Epoch 8/25
40200/40200 [==============================] - 9s 233us/step - loss: 0.4801 - acc: 0.9699 - val_loss: 0.5978 - val_acc: 0.9626
Epoch 9/25
40200/40200 [==============================] - 9s 233us/step - loss: 0.4991 - acc: 0.9688 - val_loss: 0.5357 - val_acc: 0.9666
Epoch 10/25
40200/40200 [==============================] - 9s 232us/step - loss: 0.4206 - acc: 0.9736 - val_loss: 0.4330 - val_acc: 0.9730
Epoch 11/25
40200/40200 [==============================] - 10s 238us/step - loss: 0.4163 - acc: 0.9739 - val_loss: 0.4662 - val_acc: 0.9710
Epoch 12/25
40200/40200 [==============================] - 9s 236us/step - loss: 0.4281 - acc: 0.9732 - val_loss: 0.5708 - val_acc: 0.9643
Epoch 13/25
40200/40200 [================

49998/49998 [==============================] - 11s 223us/step - loss: 0.5186 - acc: 0.9676 - val_loss: 0.5833 - val_acc: 0.9637
Epoch 13/25
49998/49998 [==============================] - 11s 224us/step - loss: 0.4144 - acc: 0.9741 - val_loss: 0.5163 - val_acc: 0.9679
Epoch 14/25
49998/49998 [==============================] - 12s 231us/step - loss: 0.4503 - acc: 0.9720 - val_loss: 0.5321 - val_acc: 0.9668
Epoch 15/25
49998/49998 [==============================] - 12s 236us/step - loss: 0.4224 - acc: 0.9737 - val_loss: 0.4366 - val_acc: 0.9728
Epoch 16/25
49998/49998 [==============================] - 11s 228us/step - loss: 0.4134 - acc: 0.9742 - val_loss: 0.5646 - val_acc: 0.9646
Epoch 17/25
49998/49998 [==============================] - 12s 235us/step - loss: 0.4334 - acc: 0.9730 - val_loss: 0.5224 - val_acc: 0.9674
Epoch 18/25
49998/49998 [==============================] - 11s 226us/step - loss: 0.4391 - acc: 0.9725 - val_loss: 0.4893 - val_acc: 0.9696
Epoch 19/25
49998/49998 [=======

50000/50000 [==============================] - 11s 229us/step - loss: 0.5261 - acc: 0.9673 - val_loss: 0.6942 - val_acc: 0.9569
Epoch 18/25
50000/50000 [==============================] - 12s 244us/step - loss: 0.5296 - acc: 0.9671 - val_loss: 0.4889 - val_acc: 0.9696
Epoch 19/25
50000/50000 [==============================] - 12s 236us/step - loss: 0.4968 - acc: 0.9691 - val_loss: 0.5944 - val_acc: 0.9629
Epoch 20/25
50000/50000 [==============================] - 11s 224us/step - loss: 0.4700 - acc: 0.9707 - val_loss: 0.4630 - val_acc: 0.9712
Epoch 21/25
50000/50000 [==============================] - 11s 227us/step - loss: 0.3928 - acc: 0.9755 - val_loss: 0.4319 - val_acc: 0.9730
Epoch 22/25
50000/50000 [==============================] - 12s 231us/step - loss: 0.4657 - acc: 0.9711 - val_loss: 0.7436 - val_acc: 0.9536612 - acc: 0
Epoch 23/25
50000/50000 [==============================] - 11s 224us/step - loss: 0.4659 - acc: 0.9710 - val_loss: 0.5374 - val_acc: 0.9665
Epoch 24/25
50000/50

Epoch 23/25
48533/48533 [==============================] - 11s 223us/step - loss: 0.4570 - acc: 0.9715 - val_loss: 0.6062 - val_acc: 0.9624
Epoch 24/25
48533/48533 [==============================] - 11s 223us/step - loss: 0.5966 - acc: 0.9629 - val_loss: 0.4961 - val_acc: 0.9691
Epoch 25/25
10000/10000 [==============================] - 1s 84us/step
0.974
(49998, 28, 28, 1)
(8333, 28, 28, 1)
(41665, 28, 28, 1)
Train on 49998 samples, validate on 19800 samples
Epoch 1/25
49998/49998 [==============================] - 11s 229us/step - loss: 0.3994 - acc: 0.9751 - val_loss: 0.5775 - val_acc: 0.9641
Epoch 2/25
49998/49998 [==============================] - 11s 222us/step - loss: 0.4165 - acc: 0.9741 - val_loss: 0.5124 - val_acc: 0.9681
Epoch 3/25
49998/49998 [==============================] - 11s 223us/step - loss: 0.3930 - acc: 0.9756 - val_loss: 0.5355 - val_acc: 0.9667
Epoch 4/25
49998/49998 [==============================] - 11s 222us/step - loss: 0.4693 - acc: 0.9708 - val_loss: 0.539

49995/49995 [==============================] - 11s 226us/step - loss: 0.4698 - acc: 0.9708 - val_loss: 0.6827 - val_acc: 0.9575
Epoch 3/25
49995/49995 [==============================] - 11s 224us/step - loss: 0.3811 - acc: 0.9763 - val_loss: 0.4634 - val_acc: 0.9711
Epoch 4/25
49995/49995 [==============================] - 11s 225us/step - loss: 0.3927 - acc: 0.9756 - val_loss: 0.5377 - val_acc: 0.9666
Epoch 5/25
49995/49995 [==============================] - 11s 224us/step - loss: 0.4418 - acc: 0.9725 - val_loss: 0.4759 - val_acc: 0.9704
Epoch 6/25
49995/49995 [==============================] - 11s 224us/step - loss: 0.3492 - acc: 0.9782 - val_loss: 0.4523 - val_acc: 0.9719
Epoch 7/25
49995/49995 [==============================] - 11s 223us/step - loss: 0.3351 - acc: 0.9792 - val_loss: 0.5134 - val_acc: 0.9680
Epoch 8/25
49995/49995 [==============================] - 11s 225us/step - loss: 0.3528 - acc: 0.9781 - val_loss: 0.4747 - val_acc: 0.9705
Epoch 9/25
49995/49995 [==============

In [ ]:
ratio_array = [ (1,2) ]
train_array = [ 150_000 ]



for num in train_array:
    for ratio in ratio_array:
        if (ratio != ratio_array[0]) & (num != train_array[0]):
            del cl
        cl = classifier()
        cl.train(totTrain=num, epochs=25, ratio=ratio)


(100000, 28, 28, 1)
(0, 28, 28, 1)
(100000, 28, 28, 1)
Train on 100000 samples, validate on 19800 samples
Epoch 1/25
100000/100000 [==============================] - 22s 216us/step - loss: 0.3479 - acc: 0.9784 - val_loss: 0.6749 - val_acc: 0.9580
Epoch 2/25
100000/100000 [==============================] - 21s 209us/step - loss: 0.3708 - acc: 0.9769 - val_loss: 0.4475 - val_acc: 0.9721
Epoch 3/25
100000/100000 [==============================] - 21s 208us/step - loss: 0.2978 - acc: 0.9815 - val_loss: 0.4881 - val_acc: 0.9697
Epoch 4/25
100000/100000 [==============================] - 21s 209us/step - loss: 0.3449 - acc: 0.9785 - val_loss: 0.4871 - val_acc: 0.9697
Epoch 5/25
100000/100000 [==============================] - 21s 209us/step - loss: 0.4548 - acc: 0.9717 - val_loss: 0.5750 - val_acc: 0.9643
Epoch 6/25
100000/100000 [==============================] - 21s 209us/step - loss: 0.3535 - acc: 0.9780 - val_loss: 0.5619 - val_acc: 0.9651
Epoch 7/25
100000/100000 [======================